## **Chapter 4: Spark SQL & Dataframe**
### ***Ex2: Flights***
Cho tập tin flights_small.csv và airports.csv
Yêu cầu:
1. Đọc tập tin flights_small.csv vào data.
2. In schema của data. Hiển thị 3 dòng đầu tiên của dữ liệu. Cho biết dữ liệu có bao nhiêu
dòng?
3. Từ data hãy tạo một view có tên là flights_small
4. Từ view flights_small hãy tạo một dataframe có tên là flights. Hiển thị 3 dòng đầu tiên của dữ
liệu
5. Trong flights, hãy tạo thêm cột duration_hrs (= air_time/60)
6. Lọc dữ liệu các chuyến bay có distance > 2000 => long_flights1. Cho biết có bao nhiêu dòng
dữ liệu thỏa điều kiện này? (bằng cả SQL và Dataframe)
7. Lọc dữ liệu các chuyến bay có 300 >= air_time >= 600 => time_flights. Cho biết có bao nhiêu
dòng dữ liệu thỏa điều kiện này? (bằng cả SQL và Dataframe)
8. Tạo dữ liệu selected1 từ flights với các cột "origin", "dest", "carrier". Tạo biến diều kiện lọc thứ
nhất filterA là các chuyến bay có origin là "SEA", tạo biến điều kiện lọc thứ hai filterB là các
chuyến bay bay có dest là "PDX". Từ đó hãy tạo Dataframe kết quả selected2 từ selected1
thỏa cả filterA và filterB. Cho biết có bao nhiêu dòng dữ liệu thỏa điều kiện trên.
9. Tạo riêng một biến avg_speed (alias "avg_speed") là tốc độ trung bình của chuyến bay (tính
theo giờ) distance/(air_time/60). Tạo Dataframe speed1 từ flights với các cột "origin", "dest",
"tailnum" và cột "avg_speed" vừa tạo
10. Tạo Dataframe speed2 từ từ flights với các cột "origin", "dest", "tailnum" và cột avg_speed =
distance/(air_time/60) trong cùng một lệnh (dùng selectExpr)
11. Sử dụng aggregation method để: tìm air_time nhỏ nhất của các chuyến bay có origin là
"PDX"; tìm distance lớn nhất của các chuyến bay có origin là "SEA"; tìm tổng duration các
chuyến bay (theo giờ).
12. Nhóm dữ liệu và đếm số chuyến bay theo từng "tailnum"; nhóm dữ liệu và đếm số chuyến bay
theo từng "origin"; nhóm dữ liệu và tính trung bình của air_time theo từng "origin"
13. Nhóm các chuyến bay theo "month", "dest" => by_month_dest. Tính trung bình "dep_delay"
theo by_month_dest. Tính std "dep_delay" theo by_month_dest.
14. Đọc tập tin airports.csv vào airports. In schema của airports. Hiển thị 3 dòng đầu tiên của dữ
liệu. Cho biết dữ liệu có bao nhiêu dòng?
15. Đổi tên cột "faa" trong airports thành "dest".
16. Tạo một Dataframe mới bằng cách kết hợp flights và airports theo "dest"

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
!tar xf spark-3.1.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql import functions as F

In [8]:
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_4'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_4


In [11]:
from pyspark import SparkContext
sc = SparkContext()


ValueError: ignored

In [12]:
spark = SparkSession(sc)

### **1. Đọc tập tin flights_small.csv vào data**

In [14]:
data = spark.read.csv("data/flights_small.csv", header=True, inferSchema=True)

### **2. In schema của data. Hiển thị 3 dòng đầu tiên của dữ liệu. Cho biết dữ liệu có bao nhiêu dòng?**

In [15]:
data.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [17]:
print("Number of rows:", data.count())

Number of rows: 10000


### **3. Từ data hãy tạo một view có tên là flights_smal**

In [18]:
data.createOrReplaceTempView("flights_small")

### **4. Từ view flights_small hãy tạo một dataframe có tên là flights. Hiển thị 3 dòng đầu tiên của dữ liệu**

In [19]:
flights = spark.table("flights_small")

In [20]:
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows



### **5. Trong flights, hãy tạo thêm cột duration_hrs (= air_time/60)**


In [21]:
flights = flights.withColumn("duration_hrs", flights.air_time/60)

In [23]:
flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        1.85|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



In [25]:
for row in flights.head(3):
    print(row)

Row(year=2014, month=12, day=8, dep_time='658', dep_delay='-7', arr_time='935', arr_delay='-5', carrier='VX', tailnum='N846VA', flight=1780, origin='SEA', dest='LAX', air_time='132', distance=954, hour='6', minute='58', duration_hrs=2.2)
Row(year=2014, month=1, day=22, dep_time='1040', dep_delay='5', arr_time='1505', arr_delay='5', carrier='AS', tailnum='N559AS', flight=851, origin='SEA', dest='HNL', air_time='360', distance=2677, hour='10', minute='40', duration_hrs=6.0)
Row(year=2014, month=3, day=9, dep_time='1443', dep_delay='-2', arr_time='1652', arr_delay='2', carrier='VX', tailnum='N847VA', flight=755, origin='SEA', dest='SFO', air_time='111', distance=679, hour='14', minute='43', duration_hrs=1.85)


### **6. Lọc dữ liệu các chuyến bay có distance > 2000 => long_flights1. Cho biết có bao nhiêu dòng dữ liệu thỏa điều kiện này? (bằng cả SQL và Dataframe)**

In [26]:
long_flights2 = flights.filter("distance > 2000")

In [27]:
print("Number of rows (distance >2000):", long_flights2.count())

Number of rows (distance >2000): 1481


In [28]:
long_flights2.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    1| 13|    2156|       -9|     607|      -15|     AS| N597AS|    24|   SEA| BOS|     290|    2496|  21|    56|4.833333333333333|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
only showing top 2 rows



In [29]:
query = '''SELECT * FROM flights_small WHERE distance > 2000'''
long_flights2_sql = spark.sql(query)

In [30]:
long_flights2_sql.count()


1481

### **7. Lọc dữ liệu các chuyến bay có 300 >= air_time >= 600 => time_flights. Cho biết có bao nhiêu dòng dữ liệu thỏa điều kiện này? (bằng cả SQL và Dataframe)**

In [31]:
time_flights = flights.filter("air_time >= 300 and air_time <= 600")
time_flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|   12|  4|     954|       -6|    1348|      -17|     HA| N395HA|    29|   SEA| OGG|     333|    2640|   9|    54|        5.55|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 2 rows



In [32]:
print("Number of rows (air_time > 300 and air_time < 600):", time_flights.count())

Number of rows (air_time > 300 and air_time < 600): 440


In [34]:
query = '''SELECT * FROM flights_small WHERE air_time between 300 and 600'''
time_flights_sql = spark.sql(query)
time_flights_sql.count()

440

### **8. Tạo dữ liệu selected1 từ flights với các cột "origin", "dest", "carrier". Tạo biến diều kiện lọc thứ nhất filterA là các chuyến bay có origin là "SEA", tạo biến điều kiện lọc thứ hai filterB là các**

In [35]:
selected1 = flights.select(flights.origin, flights.dest, flights.carrier)

In [36]:
filterA = flights.origin == "SEA"
filterB = flights.dest == "PDX"

In [39]:
selected2 = selected1.filter(filterA).filter(filterB)
selected2.show(5)

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 5 rows



In [40]:
print("Number of rows (origin = 'SEA' & dest = 'PDX'):",selected2.count())

Number of rows (origin = 'SEA' & dest = 'PDX'): 157


### **9. Tạo riêng một biến avg_speed (alias "avg_speed") là tốc độ trung bình của chuyến bay (tính theo giờ) distance/(air_time/60). Tạo Dataframe speed1 từ flights với các cột "origin", "dest", "tailnum" và cột "avg_speed" vừa tạo**

In [41]:
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

In [43]:
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)
speed1.show(2)


+------+----+-------+-----------------+
|origin|dest|tailnum|        avg_speed|
+------+----+-------+-----------------+
|   SEA| LAX| N846VA|433.6363636363636|
|   SEA| HNL| N559AS|446.1666666666667|
+------+----+-------+-----------------+
only showing top 2 rows



### **10. Tạo Dataframe speed2 từ từ flights với các cột "origin", "dest", "tailnum" và cột avg_speed = distance/(air_time/60) trong cùng một lệnh (dùng selectExpr)**

In [44]:
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")
speed2.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

### **11. Sử dụng aggregation method để: tìm air_time nhỏ nhất của các chuyến bay có origin là "PDX"; tìm distance lớn nhất của các chuyến bay có origin là "SEA"; tìm tổng duration các chuyến bay (theo giờ).**


In [46]:
from pyspark.sql.types import IntegerType

In [47]:
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

In [51]:
# air_time nhỏ nhất
flights.filter(flights.origin == "PDX").groupBy().min("air_time").show()


+-------------+
|min(air_time)|
+-------------+
|           24|
+-------------+



In [52]:
# distance lớn nhất 
flights.filter(flights.origin == "SEA").groupBy().max("distance").show()

+-------------+
|max(distance)|
+-------------+
|         2724|
+-------------+



In [50]:
# tổng giờ 
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



### **12. Nhóm dữ liệu và đếm số chuyến bay theo từng "tailnum"; nhóm dữ liệu và đếm số chuyến bay theo từng "origin"; nhóm dữ liệu và tính trung bình của air_time theo từng "origin"**


In [53]:
from pyspark.sql.functions import avg

In [55]:
by_plane = flights.groupBy("tailnum")
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [57]:
by_origin = flights.groupBy("origin").count()
by_origin.show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



In [58]:
# Average air_time 
flights.groupBy("origin").avg("air_time").show()


+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



### **13. Nhóm các chuyến bay theo "month", "dest" => by_month_dest. Tính trung bình "dep_delay" theo by_month_dest. Tính std "dep_delay" theo by_month_dest.**

In [59]:
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

In [60]:
by_month_dest = flights.groupBy("month", "dest")

In [61]:
by_month_dest.avg("dep_delay").show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    4| PHX| 1.6833333333333333|
|    1| RDM|             -1.625|
|    5| ONT| 3.5555555555555554|
|    7| OMA|               -6.5|
|    8| MDW|               7.45|
|    6| DEN|  5.418181818181818|
|    5| IAD|               -4.0|
|   12| COS|               -1.0|
|   11| ANC|  7.529411764705882|
|    5| AUS|              -0.75|
|    5| COS| 11.666666666666666|
|    2| PSP|                0.6|
|    4| ORD|0.14285714285714285|
|   10| DFW| 18.176470588235293|
|   10| DCA|               -1.5|
|    8| JNU|             18.125|
|   11| KOA|               -1.0|
|   10| OMA|-0.6666666666666666|
|    6| ONT|              9.625|
|    3| MSP|                3.2|
+-----+----+-------------------+
only showing top 20 rows



In [62]:
by_month_dest.agg(F.stddev("dep_delay")).show()

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    4| PHX|    15.003380033491737|
|    1| RDM|     8.830749846821778|
|    5| ONT|    18.895178691342874|
|    7| OMA|    2.1213203435596424|
|    8| MDW|    14.467659032985843|
|    6| DEN|    13.536905534420026|
|    5| IAD|    3.8078865529319543|
|   12| COS|    1.4142135623730951|
|   11| ANC|    18.604716401245316|
|    5| AUS|     4.031128874149275|
|    5| COS|     33.38163167571851|
|    2| PSP|     4.878524367060187|
|    4| ORD|    11.593882803741764|
|   10| DFW|     45.53019017606675|
|   10| DCA|    0.7071067811865476|
|    8| JNU|     40.79368823727514|
|   11| KOA|    1.8708286933869707|
|   10| OMA|    5.8594652770823155|
|    6| ONT|     25.98316762829351|
|    3| MSP|    21.556779370817555|
+-----+----+----------------------+
only showing top 20 rows



### **14. Đọc tập tin airports.csv vào airports. In schema của airports. Hiển thị 3 dòng đầu tiên của dữ liệu. Cho biết dữ liệu có bao nhiêu dòng?**

In [64]:
airports = spark.read.csv("data/airports.csv", header=True, inferSchema=True)

In [65]:
airports.printSchema()

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



In [66]:
print("Number of lines:", airports.count())

Number of lines: 1397


In [67]:
airports.show(3)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



### **5. Đổi tên cột "faa" trong airports thành "dest".**

In [68]:
airports = airports.withColumnRenamed("faa", "dest")
airports.show(5)

+----+--------------------+----------+-----------+----+---+---+
|dest|                name|       lat|        lon| alt| tz|dst|
+----+--------------------+----------+-----------+----+---+---+
| 04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
| 06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
| 09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+----+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



### **16. Tạo một Dataframe mới bằng cách kết hợp flights và airports theo "dest"**

In [69]:
flights_with_airports = flights.join(airports, on="dest", how="leftouter")
flights_with_airports.show(3)

+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------+------------------+---------+-----------+---+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|duration_hrs|              name|      lat|        lon|alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------+------------------+---------+-----------+---+---+---+
| LAX|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|     132|     954|   6|    58|         2.2|  Los Angeles Intl|33.942536|-118.408075|126| -8|  A|
| HNL|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|     360|    2677|  10|    40|         6.0|     Honolulu Intl|21.318681|-157.922428| 13|-10|  N|
| SFO|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N8